In [10]:
import time
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [11]:
key = "nyVvF841UBPdCn%2BvugXzKE1EEwkhTBIhjhaXOQgC%2FAlD2F0YongS3tfgXgIxbW5lmlkuJ6Szjamn18IPVNMpXA%3D%3D"

In [12]:
def getArrivalInfo(arsId):
    html = requests.get('http://ws.bus.go.kr/api/rest/stationinfo/getStationByUid?serviceKey='+key+'&arsId='+arsId).text
    root = BeautifulSoup(html, 'xml')
    station_name = root.find('stNm').text
    result = f'======= {station_name} =======\n'
    items = root.find_all('itemList')
    for item in items:
        bus_route_id = item.find('busRouteId').text
        bus_route_name = item.find('rtNm').text
        predict_msg = item.find('arrmsg1').text  
        if predict_msg == '운행종료':
            result += f'버스번호: {bus_route_name}: 운행종료\n'
        elif predict_msg == '곧 도착':
            result += f'버스번호: {bus_route_name}: 곧 도착\n'
        else:
            time, suffix = predict_msg.split('후')
            time = time.strip()
            minutes, seconds = time.split('분')
            result += f'버스번호: {bus_route_name}: {minutes.strip()} 분 {seconds} 후\n'
    return result

In [13]:
import serial

In [14]:
# serial.Serial("/dev/tty.usbserial-1130",9600)
serial.Serial("/dev/tty.usbserial-0001",115200)
# ard = serial.Serial("/dev/tty.usbserial-1130",9600)
ard = serial.Serial("/dev/tty.usbserial-0001",115200)

In [15]:
def main():
    arsIds = ['05251', '05698', '05699']
    while True:
        for arsId in arsIds:
            # 시리얼 통신을 통해 아두이노로 정보를 전송
            ard.write(getArrivalInfo(arsId).encode())
            time.sleep(1)
        time.sleep(59)  # API 호출 시간 간격을 고려하여 59초를 대기

In [17]:
if __name__ == "__main__":
    main()

KeyboardInterrupt: 